In [0]:
import torch
import pdb
import torch.nn as nn
import math
from torch.autograd import Variable
from torch.autograd import Function
import time
from torch.distributions.relaxed_bernoulli import RelaxedBernoulli,LogitRelaxedBernoulli
import numpy as np


def Binarize(tensor,quant_mode='det'):
    if quant_mode=='det':
      tensor = tensor.sign()
#       tensor[tensor==0] = 1
      return tensor
    else:
        return tensor.add_(1).div_(2).add_(torch.rand(tensor.size()).add(-0.5)).clamp_(0,1).round().mul_(2).add_(-1)

def sample2(mu, log_sigma2):
    eps = torch.randn_like(mu)
    s = mu + torch.exp(log_sigma2 / 2) * eps
    return s
  
  
def sample_gumbel(shape, eps=1e-20):
    unif = torch.rand(*shape).cuda()
    g = -torch.log(-torch.log(unif + eps))
    return g

def sample_gumbel_softmax(logits, temperature):
    """
        Input:
        logits: Tensor of log probs, shape = BS x k
        temperature = scalar
        
        Output: Tensor of values sampled from Gumbel softmax.
                These will tend towards a one-hot representation in the limit of temp -> 0
                shape = BS x k
    """
    g = sample_gumbel(logits.shape)
    h = (g + logits)/temperature
    h_max = h.max(dim=-1, keepdim=True)[0]
    h = h - h_max
    cache = torch.exp(h)
    y = cache / cache.sum(dim=-1, keepdim=True)
    return y
  
def sampling(mu,sig):
  x = Normal(mu,sig)
#   x = x.sample(torch.tensor([out_features]))
#   print(x.cdf)
  p = 1 - x.cdf(0)
#   print((x.cdf(0))[0])
#   p = Binarize(p)
#   print(p[0])
#   a = ((p+1)/2).bernoulli()
#   a = a*2-1
# #   print(a[0])
#   a = torch.nn.functional.gumbel_softmax(p, tau=1, hard=True, eps=1e-10, dim=-1)
#   
#   l = LogitRelaxedBernoulli(torch.tensor([1.]).cuda(),p)
#   l = l.sample()
#   a = sample_gumbel_softmax(p,1.0)
#   print(x[0]) 
  return p



import torch.nn._functions as tnnf



    
class PBinarizeLinear(nn.Linear):

    def __init__(self, *kargs, **kwargs):
        super(PBinarizeLinear, self).__init__(*kargs, **kwargs)
#         w = torch.empty_like(self.weight)
#         self.weight.data = nn.init.uniform_(w,-1,1)
#         theta.requires_grad_
#         self.weight.data = ((theta+1)/2).bernoulli()
#         self.weight.data = Binarize(self.weight.data-0.5)
#         self.weight.data = Binarize(theta)
        

    def forward(self, input):
#         print(input.data[0])
      
      
        if not hasattr(self.weight,'org'):
            self.weight.org=self.weight.data.clone()  
            
        self.weight.data=Binarize(self.weight.org)
#         print(self.weight.data)
#         print(self.weight.org)
#         theta = self.weight
        theta = torch.tanh(self.weight)
#         print(theta)
#         print(input[0])
        

#         print(input[0])
        if input.size(1) != 784:
          mu = nn.functional.linear(input,theta)
          left = input**2 - (1- input**2)
          right = theta**2 - (1-theta**2)
          sigma = 1 - nn.functional.linear(left,right)
        else:
#           print((input**2)[0])
#           print((1-(theta**2))[0])
          mu = nn.functional.linear(input,theta)       
          sigma = nn.functional.linear(input**2,1-(theta**2))
        
#         
#         print(mu.shape)
        m = mu.mean(0,True)
        
        v = sigma.var(0,True)
     
        mu = 0.5*(mu-m)/((v+(0.0001)).sqrt()+0.5)
        sigma = 0.5**2*sigma/(v+0.0001)

        
       
        out1 = sampling(mu,sigma)

        if self.out_features==10:
          return mu
        else:
          return out1




class PBinarizeConv2d(nn.Conv2d):

    def __init__(self, *kargs, **kwargs):
        super(PBinarizeConv2d, self).__init__(*kargs, **kwargs)
        

    def forward(self, input):
      
        if not hasattr(self.weight,'org'):
            self.weight.org=self.weight.data.clone()  
            
        self.weight.data=Binarize(self.weight.org)
        
        theta = torch.tanh(self.weight)
        

        if input.size(1) != 3:
          mu = nn.functional.conv2d(input, theta, None, self.stride,
                                   self.padding, self.dilation, self.groups)
          left = input**2 - (1- input**2)
          right = theta**2 - (1-theta**2)
          sigma = 1 - nn.functional.conv2d(left, right, None, self.stride,
                                   self.padding, self.dilation, self.groups)
        else:
          mu = nn.functional.conv2d(input, theta, None, self.stride,
                                   self.padding, self.dilation, self.groups)
          sigma = nn.functional.conv2d(input**2, 1-(theta**2), None, self.stride,
                                   self.padding, self.dilation, self.groups)
        
#         print(mu.shape)
        m = mu.mean((0,2,3),True)
        
        v = sigma.var((0,2,3)).unsqueeze(0).unsqueeze(-1).unsqueeze(-1)
     
        mu = 0.5*(mu-m)/((v+(0.0001)).sqrt()+0.5)
        sigma = 0.5**2*sigma/(v+0.0001)
        
        
        eps = torch.randn_like(mu)
        s = mu + torch.exp(sigma / 2) * eps
        
        
        mp, indices = nn.functional.max_pool2d(s, 2, 2,0, 1, False,True)
                            
        
#         m = nn.MaxPool2d(kernel_size=2, stride=2,return_indices=True)
#         mp, indices = m(s)
        indices = indices.view(-1)
        
        mu2 = (mu.view(-1)).index_select(0,indices)  
        mu2 = mu2.view(mp.size())
        
        sigma2 = (sigma.view(-1)).index_select(0,indices)  
        sigma2 = sigma2.view(mp.size())

        
        
        out1 = sampling(mu,sigma)


#         if not self.bias is None:
#             self.bias.org=self.bias.data.clone()
#             out += self.bias.view(1, -1, 1, 1).expand_as(out)
        
        
        return out1

    



In [2]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from tqdm import tqdm
from torch.distributions.normal import Normal
from torch.distributions.relaxed_bernoulli import RelaxedBernoulli
from torch.distributions.relaxed_categorical import RelaxedOneHotCategorical

from torch.distributions.categorical import Categorical

torch.manual_seed(1)
# if args.cuda:
#     torch.cuda.manual_seed(args.seed)


# kwargs = {'num_workers': 1, 'pin_memory': True} if args.cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=128, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=128, shuffle=True)





# 32C3 - MP2 - 64C3 - Mp2 - 512FC - SM10c
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = PBinarizeConv2d(1, 32, kernel_size=3)
#         self.mp1= nn.MaxPool2d(kernel_size=2, stride=2)
      
        self.conv2 = PBinarizeConv2d(32, 64, kernel_size=3)
#         self.mp2= nn.MaxPool2d(kernel_size=2, stride=2)
     
        self.fc1 = PBinarizeLinear(36864, 512)

        
        self.fc2 = PBinarizeLinear(512, 10)


    # 32C3 - MP2 - 64C3 - Mp2 - 512FC - SM10c
  
    def forward(self, x):
      
#       print(x.shape)
        
      x = self.conv1(x)
      x = self.conv2(x)

      x = x.view(x.size(0), -1)
#         print(x.size())

      x = self.fc1(x)

      x = self.fc2(x)


      return x
  

model = Net()

print(model)

torch.cuda.device('cuda')
model.cuda()



criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)


def train(epoch):
    model.train()
    
    losses = []
    trainloader = tqdm(train_loader)
    
    for batch_idx, (data, target) in enumerate(trainloader):
 
        data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
#         print(output)
#         output = output+1e-10
        loss = criterion(output, target)

#         print(loss)

#         if epoch%40==0:
#             optimizer.param_groups[0]['lr']=optimizer.param_groups[0]['lr']*0.1

#         optimizer.zero_grad()
#         
        loss.backward()
    
        for p in list(model.parameters()):
            if hasattr(p,'org'):
                p.data.copy_(p.org)
        optimizer.step()
        
        for p in list(model.parameters()):
            if hasattr(p,'org'):
                p.org.copy_(p.data.clamp_(-0.9,0.9))
    
        losses.append(loss.item())
        trainloader.set_postfix(loss=np.mean(losses), epoch=epoch)



def test():
    model.eval()
    test_loss = 0
    correct = 0
    testloader = tqdm(test_loader)
    for data, target in testloader:
        data, target = data.cuda(), target.cuda()
        with torch.no_grad():
          data = Variable(data)
        target = Variable(target)
        output = model(data)
        test_loss += criterion(output, target).item() # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
        
        

        testloader.set_postfix(loss=test_loss / len(test_loader.dataset),acc=str((100. *correct / len(test_loader.dataset)).numpy())+'%')
    
    test_loss /= len(test_loader.dataset)
    
    




Net(
  (conv1): PBinarizeConv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): PBinarizeConv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (fc1): PBinarizeLinear(in_features=36864, out_features=512, bias=True)
  (fc2): PBinarizeLinear(in_features=512, out_features=10, bias=True)
)


In [3]:
%%%time
for epoch in range(20):
    train(epoch)
    test()

100%|██████████| 79/79 [00:02<00:00, 27.96it/s, acc=96%, loss=0.000817]

CPU times: user 9min 11s, sys: 3min 46s, total: 12min 58s
Wall time: 13min 22s


In [0]:
a = torch.rand(4,3,12,12)
# a

In [5]:
(a.var((0,2,3))).size()

torch.Size([3])

In [0]:
m = nn.MaxPool2d(kernel_size=2, stride=2,return_indices=True)

In [0]:
c = m(a)

In [8]:
c[1].size()

torch.Size([4, 3, 6, 6])

In [0]:
b = torch.randn(4,3,12,12)

In [0]:
z = b.view(-1)

In [0]:
y = c[1].view(-1)

In [12]:
y.size()

torch.Size([432])

In [0]:
g = z.index_select(0,y)

In [14]:
g.view(c[1].size())

tensor([[[[-1.2636e+00,  9.6282e-01,  2.3601e-02, -1.5993e+00, -9.1462e-01,
           -5.3467e-01],
          [ 1.9239e+00,  1.2761e+00, -6.8907e-01, -4.9525e-01, -7.7302e-01,
           -4.4493e-01],
          [-7.0929e-01, -1.0349e-03, -3.2382e-01, -4.6858e-01, -5.7909e-01,
            4.0405e-01],
          [-6.6826e-01, -5.9336e-01, -1.5763e+00,  1.3286e+00, -8.7460e-02,
           -1.6614e-01],
          [ 6.3922e-01, -6.1810e-01, -1.5021e-01,  3.4608e-02,  1.0347e+00,
           -8.9240e-01],
          [-6.7547e-01,  6.2669e-01,  6.4222e-01,  1.1431e+00,  1.0325e+00,
           -9.1363e-01]],

         [[-1.5695e+00, -5.2523e-01,  2.3601e-02, -4.9735e-01, -4.3570e-01,
           -1.5464e+00],
          [-1.5172e+00, -6.7131e-02,  6.9641e-01, -4.9525e-01, -7.7302e-01,
            6.1572e-01],
          [-5.6152e-01, -1.0349e-03, -3.2382e-01,  5.3469e-01, -5.7909e-01,
            4.0405e-01],
          [-5.0737e-01, -6.7643e-02,  2.6314e+00,  1.3286e+00, -4.7099e-01,
           -1

In [15]:
(a.var((0,2,3),True)).size()

torch.Size([3])

In [16]:
(a.var((0)).unsqueeze(-1)).size()

torch.Size([3, 12, 12, 1])

In [17]:
torch.ones_like(a)

tensor([[[[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]],

         [[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]],

         [[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]]],


        [[[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
        

In [18]:
a.bernoulli()

tensor([[[[0., 0., 0.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [0., 0., 0.,  ..., 0., 1., 0.],
          ...,
          [1., 0., 1.,  ..., 1., 0., 1.],
          [0., 0., 0.,  ..., 1., 0., 0.],
          [0., 1., 0.,  ..., 1., 1., 0.]],

         [[1., 0., 1.,  ..., 0., 1., 1.],
          [0., 0., 0.,  ..., 1., 1., 0.],
          [1., 0., 0.,  ..., 0., 0., 1.],
          ...,
          [1., 0., 0.,  ..., 0., 0., 1.],
          [0., 1., 0.,  ..., 1., 1., 1.],
          [0., 1., 1.,  ..., 1., 0., 1.]],

         [[0., 0., 1.,  ..., 1., 1., 1.],
          [1., 1., 0.,  ..., 0., 1., 0.],
          [1., 1., 0.,  ..., 0., 1., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 1., 0.],
          [1., 1., 1.,  ..., 1., 0., 1.],
          [0., 0., 1.,  ..., 0., 1., 1.]]],


        [[[1., 1., 0.,  ..., 0., 0., 0.],
          [0., 1., 1.,  ..., 0., 0., 1.],
          [1., 1., 0.,  ..., 0., 0., 1.],
          ...,
          [1., 0., 1.,  ..., 1., 0., 1.],
        

In [19]:
a=Binarize(a)
a

tensor([[[[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]],

         [[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]],

         [[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]]],


        [[[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
        

In [20]:
torch.nn.functional.gumbel_softmax(a, tau=1, hard=True, eps=1e-10, dim=-1)

tensor([[[[0., 0., 0.,  ..., 1., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 1., 0.],
          ...,
          [0., 0., 0.,  ..., 1., 0., 0.],
          [0., 1., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         [[0., 0., 0.,  ..., 0., 1., 0.],
          [0., 0., 0.,  ..., 0., 0., 1.],
          [0., 0., 1.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 1., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [1., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 1.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        [[[0., 0., 1.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
        

In [21]:
(a+1)/2

tensor([[[[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]],

         [[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]],

         [[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]]],


        [[[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
        

In [22]:
a.tanh()

tensor([[[[0.7616, 0.7616, 0.7616,  ..., 0.7616, 0.7616, 0.7616],
          [0.7616, 0.7616, 0.7616,  ..., 0.7616, 0.7616, 0.7616],
          [0.7616, 0.7616, 0.7616,  ..., 0.7616, 0.7616, 0.7616],
          ...,
          [0.7616, 0.7616, 0.7616,  ..., 0.7616, 0.7616, 0.7616],
          [0.7616, 0.7616, 0.7616,  ..., 0.7616, 0.7616, 0.7616],
          [0.7616, 0.7616, 0.7616,  ..., 0.7616, 0.7616, 0.7616]],

         [[0.7616, 0.7616, 0.7616,  ..., 0.7616, 0.7616, 0.7616],
          [0.7616, 0.7616, 0.7616,  ..., 0.7616, 0.7616, 0.7616],
          [0.7616, 0.7616, 0.7616,  ..., 0.7616, 0.7616, 0.7616],
          ...,
          [0.7616, 0.7616, 0.7616,  ..., 0.7616, 0.7616, 0.7616],
          [0.7616, 0.7616, 0.7616,  ..., 0.7616, 0.7616, 0.7616],
          [0.7616, 0.7616, 0.7616,  ..., 0.7616, 0.7616, 0.7616]],

         [[0.7616, 0.7616, 0.7616,  ..., 0.7616, 0.7616, 0.7616],
          [0.7616, 0.7616, 0.7616,  ..., 0.7616, 0.7616, 0.7616],
          [0.7616, 0.7616, 0.7616,  ..., 0

In [23]:
a.mean(0)

tensor([[[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]],

        [[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 

In [0]:
p= np.count_nonzero((a+1)/2,axis=0)/np.count_nonzero(a,axis=0)

In [25]:
1-(p)**2

array([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0.,

In [26]:
np.count_nonzero((a+1)/2)

1728

In [0]:
mu = torch.randn(5)
sig = torch.randn(5)


In [28]:
x=Normal(mu,sig)
1 - x.cdf(0)

tensor([0.0025, 0.9992, 0.4764, 0.4444, 0.2710])

In [0]:
m = -4.6
v = 25936
x = Normal(m,v)
p = 1 - x.cdf(0)
# s = sample_gumbel_softmax(p,1.0)



In [30]:
w = torch.empty(3, 5)
nn.init.uniform_(w,-1,1)

tensor([[ 0.1520,  0.3393,  0.6580,  0.8372, -0.4762],
        [-0.2452, -0.0465,  0.0222,  0.7341,  0.5384],
        [-0.0112,  0.0451,  0.0851, -0.1766,  0.7771]])

In [31]:
p.sample()

AttributeError: ignored

In [0]:
aa = Normal(m,v)
# aa.sample(torch.tensor([20]))

In [0]:
1 - x.cdf(0)

In [0]:
m = RelaxedOneHotCategorical(torch.tensor([1.]),a)
m.sample()

In [0]:
""# # Execute this code block to install dependencies when running on colab
# try:
#     import torch
# except:
#     from os.path import exists
#     from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
#     platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
#     cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
#     accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

#     !pip install -q http://download.pytorch.org/whl/{accelerator}/torch-1.0.0-{platform}-linux_x86_64.whl torchvision

# try: 
#     import torchbearer
# except:
#     !pip install torchbearer
    
# from torchbearer import Trial
# torchbearer_trial = Trial(model, optimizer, criterion, metrics=['loss', 'accuracy']).to('cuda:0')
# torchbearer_trial.with_generators(train_loader, test_generator=test_loader)
# torchbearer_trial.run(epochs=5)